In [ ]:
%pip install kafka-python
%pip install pymongo

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 507 kB 21.8 MB/s 
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Setup
Import required libraries.

In [ ]:
import logging as log
import pymongo
from kafka import KafkaConsumer
from kafka.errors import KafkaError

### push_to_mongodb

This function pushes a dictionary of weather forecast data to the MongoDB database.

##### Arguments
    message : dict
        The forecast data as received from Kafka.
        Must include the keys 'today' and '7 days', which should each link to a dictionary of forecast data for the corresponding day.

##### Returns
    N/A

##### Side effects
    The forecast data is inserted into the MongoDB database.


In [ ]:
def push_to_mongodb(message):
    client = pymongo.MongoClient('mongodb://localhost:0001/')
    db = client['weather_forecasts']

    for key in message: # 'today' or '7 days'
        value = {
            message[key]['date']: {
                weather_quantity: weather_value
                    for weather_quantity, weather_value in message[key].items()
                    if weather_quantity != 'date'
            }
        }

        col = db[key]
        col.insert_one(value)

### Migrate data
Listen to the 'forecast' topic on the Kafka platform. If there are any new messages, push them to the MongoDB database.

In [ ]:
try:
    consumer = KafkaConsumer('forecast', bootstrap_servers='localhost:0000')
    for message in consumer:
        push_to_mongodb(message)
except KafkaError:
    log.exception('Receive error')
    pass

ERROR:root:Receive error
Traceback (most recent call last):
  File "<ipython-input-5-1a5293953853>", line 2, in <module>
    consumer = KafkaConsumer('forecast', bootstrap_servers='localhost:0000')
  File "/usr/local/lib/python3.7/site-packages/kafka/consumer/group.py", line 356, in __init__
    self._client = KafkaClient(metrics=self._metrics, **self.config)
  File "/usr/local/lib/python3.7/site-packages/kafka/client_async.py", line 244, in __init__
    self.config['api_version'] = self.check_version(timeout=check_timeout)
  File "/usr/local/lib/python3.7/site-packages/kafka/client_async.py", line 900, in check_version
    raise Errors.NoBrokersAvailable()
kafka.errors.NoBrokersAvailable: NoBrokersAvailable


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa73dd04-f36c-4056-b6c3-896407612102' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>